## hetnet machine learning

In [2]:
import itertools
import collections
import csv
import gzip

import pandas
import sklearn.metrics

import hetio.readwrite
import hetio.pathtools

/home/dhimmels/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


## Download and load hetnet

In [3]:
# # Download network
# url = 'https://raw.githubusercontent.com/dhimmel/integrate/1a43ef2718283cbfc9ae869aee1856bf20e2ad6e/data/hetnet.json.gz'
# ! wget --no-verbose --directory-prefix download/ {url}

2015-10-30 15:35:38 URL:https://raw.githubusercontent.com/dhimmel/integrate/1a43ef2718283cbfc9ae869aee1856bf20e2ad6e/data/hetnet.json.gz [41786568/41786568] -> "download/hetnet.json.gz" [1]


In [3]:
%%time
graph = hetio.readwrite.read_json('download/graph.json.gz')
metagraph = graph.metagraph

CPU times: user 2min 24s, sys: 17.7 s, total: 2min 41s
Wall time: 2min 41s


In [31]:
metapaths = metagraph.extract_metapaths('compound', 'disease', max_length=3)
metapaths.pop(0)
len(metapaths)

261

## Helper functions

In [9]:
def compute_dwpc(graph, compound_id, disease_id, metapath):
    # Compute edge exclusions
    exclude_edges = set()
    edge_id = compound_id, disease_id, 'indication', 'both'
    indication = graph.edge_dict.get(edge_id)
    if indication:
        exclude_edges.add(indication)
        exclude_edges.add(indication.inverse)

    compound = graph.get_node(('compound', compound_id))
    disease = graph.get_node(('disease', disease_id))

    paths = hetio.pathtools.paths_between(graph, compound, disease, metapath, exclude_edges=exclude_edges)
    dwpc = hetio.pathtools.DWPC(paths, damping_exponent=0.4, exclude_edges=exclude_edges)
    
    row = collections.OrderedDict()
    row['compound_id'] = compound_id
    row['disease_id'] = disease_id
    row['metapath'] = str(metapath)
    row['path_count'] = len(paths)
    row['dwpc'] = dwpc
    
    return row

## Read part_df

In [10]:
part_df = pandas.read_table('data/partition.tsv.gz')
part_df = part_df.query('indication == 1 or percentile <= 0.02')
part_df.head()

,compound_id,compound_name,disease_id,disease_name,indication,percentile
9,DB00014,Goserelin,DOID:10283,Prostate cancer,1,0.705331
20,DB00014,Goserelin,DOID:11119,Gilles de la Tourette syndrome,0,0.007863
54,DB00014,Goserelin,DOID:14268,sclerosing cholangitis,0,0.011764
58,DB00014,Goserelin,DOID:1612,Breast cancer,1,0.522334
82,DB00014,Goserelin,DOID:2986,IgA glomerulonephritis,0,0.013587


In [11]:
len(part_df)

5613

In [12]:
metanode_to_nodes = graph.get_metanode_to_nodes()

In [13]:
#compounds = metanode_to_nodes[metagraph.get_node('compound')]
#diseases = metanode_to_nodes[metagraph.get_node('disease')]
#MS = graph.get_node(('disease', 'DOID:2377'))

In [32]:
write_file = gzip.open('data/dwpc.tsv.gz', 'wt')
header = ['compound_id', 'disease_id', 'metapath', 'path_count', 'dwpc']
writer = csv.DictWriter(write_file, delimiter='\t', fieldnames=header)
writer.writeheader()
for i, row in part_df.iterrows():
    print('Computing DWPCs for {} and {}'.format(row.compound_name, row.disease_name), end='\r')
    for metapath in metapaths:
        compound_id = row['compound_id']
        disease_id = row['disease_id']
        row = compute_dwpc(graph, compound_id, disease_id, metapath)
        writer.writerow(row)

write_file.close()

# Performance

In [33]:
dwpc_df = pandas.read_table('data/dwpc.tsv.gz')
dwpc_df.tail()

,compound_id,disease_id,metapath,path_count,dwpc
1464988,DB09028,DOID:8986,CtGku>GvD,0,0
1464989,DB09028,DOID:8986,CtGku>GaD,0,0
1464990,DB09028,DOID:8986,CtGku>GuD,0,0
1464991,DB09028,DOID:8986,CtGku>GdD,0,0
1464992,DB09028,DOID:8986,CtGdDsD,0,0


In [34]:
def compute_metrics(df):
    y_true = df.indication
    y_score = df.dwpc
    series = pandas.Series()
    series['nonzero'] = (y_score > 0).mean()
    series['auroc'] = sklearn.metrics.roc_auc_score(y_true, y_score)
    series['auprc'] = sklearn.metrics.average_precision_score(y_true, y_score)
    return series

merged_df = part_df.merge(dwpc_df)
auc_df = merged_df.groupby('metapath').apply(compute_metrics).reset_index()

In [35]:
auc_df = auc_df.sort('auroc', ascending=False)
auc_df.to_csv('data/auc.tsv', sep='\t', index=False, float_format='%.3f')
auc_df

,metapath,nonzero,auroc,auprc
55,CcSEcCiD,0.590059,0.896982,0.798138
119,CiDiCiD,0.254588,0.840294,0.813803
111,CiDaGaD,0.411366,0.819927,0.740502
195,CtGtCiD,0.227151,0.807357,0.760547
121,CiDpSpD,0.409763,0.801344,0.691862
120,CiDlAlD,0.400499,0.797315,0.674678
123,CiDsDsD,0.215927,0.764094,0.689032
142,CsCsCiD,0.197933,0.753998,0.716095
169,CtGbCiD,0.182612,0.744426,0.691644
140,CsCiD,0.148762,0.741553,0.732153


In [30]:
metagraph.kind_to_abbrev

{'anatomy': 'A',
 'association': 'a',
 'binding': 'b',
 'biological process': 'BP',
 'causation': 'c',
 'cellular component': 'CC',
 'compound': 'C',
 'disease': 'D',
 'downregulation': 'd',
 'evolution': 'e',
 'expression': 'e',
 'function': 'f',
 'gene': 'G',
 'indication': 'i',
 'interaction': 'i',
 'knockdown downregulation': 'kd',
 'knockdown upregulation': 'ku',
 'localization': 'l',
 'molecular function': 'MF',
 'overexpression downregulation': 'od',
 'overexpression upregulation': 'up',
 'participation': 'p',
 'pathway': 'PW',
 'perturbation': 'PB',
 'presense': 'p',
 'regulation': 'r',
 'side effect': 'SE',
 'similarity': 's',
 'symptom': 'S',
 'target': 't',
 'upregulation': 'u',
 'variation': 'v'}